In [ ]:
from timeit import default_timer as time
import pandas as pd
import scipy.stats
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import calendar

import gc
from sklearn.ensemble import RandomForestClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb

%matplotlib inline
plt.rcParams.update({'figure.figsize':(12,6)})
plt.rcParams.update({'font.size':20})

# Feaeture Selection

In [99]:
# set some parameters
num_files = 1
num_filter = 100
num_wrapper = 30
# keep balance = 0 when num_files > 1 until I figure out this bug. Maybe somebody else can figure it out.
balance = 0
detect_rate = .03
index_name = 'record'
y_name = 'fraud_label'
good_label = 0
bad_label = 1

In [ ]:
%%time
file_name = "vars2.csv"
df = pd.read_csv(file_name)
df = df[df[index_name] <= 833508]
df = df[df[index_name] >= 38512]
df['RANDOM'] = np.random.ranf(len(df)) 
goods = df[df[y_name] == good_label]
bads = df[df[y_name] == bad_label]
del df # don't need this file anymore
num_goods = len(goods)
num_bads = len(bads)
num_vars = len(bads.columns)-2
df_sampled = pd.concat([goods,bads])
df_sampled.set_index(index_name, inplace=True)
df_sampled.sort_index(inplace=True)
Y = pd.DataFrame(df_sampled[y_name], index=df_sampled.index)
filter_score = pd.DataFrame(np.zeros((num_vars+1,2)))
filter_score.columns = ['variable','score']   
j = 0
for column in df_sampled:
    filter_score.loc[j,'variable'] = column
    filter_score.loc[j,'score'] = sps.ks_2samp(goods[column],bads[column])[0]
    j = j+1
    
filter_score.sort_values(by=['score'], ascending=False, inplace=True)
vars_keep = list(filter_score['variable'][1:num_filter+1])
print(filter_score.head(num_filter))
df_top = df_sampled.filter(vars_keep, axis=1)
del goods # delete these before starting the next file, if any
del bads
gc.collect()

In [ ]:
filter_score.sort_values(by=['score'], ascending=False, inplace=True)

In [ ]:
filter_score.head(30)

In [ ]:
filter_score.tail(10)

In [ ]:
filter_score.shape

In [ ]:
filter_score.sort_values(by=['score'], ascending=False, inplace=True)
vars_keep = list(filter_score['variable'][num_files:num_filter+3])
print(1,' vars_keep:',vars_keep)

In [ ]:
df_keep = df_top.filter(vars_keep, axis=1)
df_keep.head()

In [ ]:
df_keep.shape

In [ ]:
Y.head()

In [ ]:
Y = Y.values
X = df_keep

In [ ]:
print('time to here:', dt.datetime.now() - start_time)

In [ ]:
print(X.shape,Y.shape)

In [ ]:
def fdr(classifier, x, y, cutoff=detect_rate):
# Calculates FDR score for the given classifier on dataset x and y with cutoff value
# get the probability list from the given classifier
    return fdr_prob(y, classifier.predict_proba(x), cutoff)
def fdr_prob(y, y_prob, cutoff=detect_rate):
    if len(y_prob.shape) != 1:    # sometimes the proba list can contain many columns, one for each category
        y_prob = y_prob[:, -1:]   # only the last one (fraud_label==1) is used here.
    num_fraud = len(y[y == 1])    # count the total nunber of frauds   
# sort the proba list from high to low while retain the true (not predicted) fraud label
    sorted_prob = np.asarray(sorted(zip(y_prob, y), key=lambda x: x[0], reverse=True))
    cutoff_bin = sorted_prob[0:int(len(y) * cutoff), 1:]  # 3% cutoff
# return the FDR score (#fraud_in_cutoff / #total_fraud)
    return len(cutoff_bin[cutoff_bin == 1]) / num_fraud 

In [ ]:
%%time
# If you're doing forward selection it's enough to stop at num_wrapper variables. 
# If you're doing backward selection you need to go through all the variables to get 
# a sorted list of num_wrapper variables.
nfeatures = len(X.columns)
clf = RandomForestClassifier(n_estimators=5, n_jobs=-1) # simple, fast nonlinear model for the wrapper
sfs = SFS(clf,k_features=num_wrapper,forward=True,verbose=2,scoring=fdr,cv=2,n_jobs=-1) # forward selection
# sfs = SFS(clf,k_features=1,forward=False,verbose=2,scoring=fdr,cv=2,n_jobs=-1) # backward selection
sfs.fit(X,Y)

In [ ]:
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(sfs.get_metric_dict(),kind='std_dev', figsize=(15, 6))
plt.xticks(np.arange(0, len(X.columns), step=5))
plt.ylim([0., .7])
plt.xlim(0,num_wrapper)
plt.title('Stepwise Selection')
plt.grid()
plt.savefig('FS_fdr.png')
plt.show()

In [ ]:
vars_FS = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
vars_FS

In [ ]:
vars_final = list(vars_FS.loc[num_wrapper, 'feature_names'])
vars_final

In [ ]:
vars_FS.to_csv('VarsFS.csv', index=False)

In [ ]:
sfs.subsets_

In [ ]:
vars_final.append(index_name)
vars_fin mal.append(y_name)
vars_final

In [ ]:
X_final = df_keep.filter(vars_final, axis=1)
X_final.shape

In [ ]:
X_final.to_csv('vars_final.csv', index=False)

In [ ]:
print('time to here:', dt.datetime.now() - start_time)

In [ ]:
top30=[]
for i in vars_FS['feature_names']:
    for j in list(i):
        if j in top30:
            continue
        else:
            top30.append(j)
top30

In [ ]:
import pandas as pd
file_name = "vars2.csv"
df = pd.read_csv(file_name)
df.shape

In [ ]:
vars_final=['fulladdress_count_30',
 'ssn_lastname_count_30',
 'homephone_count_3',
 'ssn_count_14',
 'ssn_dob_count_0_by_30',
 'ssn_count_0_by_14',
 'fulladdress_homephone_count_7',
 'name_dob_count_14',
 'ssn_dob_count_7',
 'fulladdress_homephone_count_30',
 'ssn_count_0_by_30',
 'ssn_count_30',
 'ssn_count_7',
 'ssn_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_0_by_14',
 'ssn_name_dob_count_0_by_14',
 'ssn_name_dob_count_14',
 'name_dob_count_0_by_14',
 'ssn_dob_count_30',
 'name_dob_count_7',
 'ssn_name_dob_count_0_by_30',
 'name_dob_count_0_by_30',
 'ssn_dob_count_0_by_14',
 'ssn_firstname_count_7',
 'ssn_name_dob_count_30',
 'ssn_firstname_count_14',
 'ssn_lastname_count_0_by_14',
 'ssn_firstname_count_0_by_14',
 'name_dob_count_30',
 'record',
 'fraud_label']

In [ ]:
X_final = df.filter(vars_final, axis=1)
X_final.shape

In [ ]:
X_final.to_csv('vars_final.csv', index=False)

In [ ]:
# set some parameters
num_files = 1
num_filter = 100
num_wrapper = 30
# keep balance = 0 when num_files > 1 until I figure out this bug. Maybe somebody else can figure it out.
balance = 0
detect_rate = .03
index_name = 'record'
y_name = 'fraud_label'
good_label = 0
bad_label = 1

%%time
file_name = "vars2.csv"
df = pd.read_csv(file_name)
df = df[df[index_name] <= 833508]
df = df[df[index_name] >= 38512]
df['RANDOM'] = np.random.ranf(len(df)) 
goods = df[df[y_name] == good_label]
bads = df[df[y_name] == bad_label]
del df # don't need this file anymore
num_goods = len(goods)
num_bads = len(bads)
num_vars = len(bads.columns)-2
df_sampled = pd.concat([goods,bads])
df_sampled.set_index(index_name, inplace=True)
df_sampled.sort_index(inplace=True)
Y = pd.DataFrame(df_sampled[y_name], index=df_sampled.index)
filter_score = pd.DataFrame(np.zeros((num_vars+1,2)))
filter_score.columns = ['variable','score']   
j = 0
for column in df_sampled:
    filter_score.loc[j,'variable'] = column
    filter_score.loc[j,'score'] = sps.ks_2samp(goods[column],bads[column])[0]
    j = j+1
    
filter_score.sort_values(by=['score'], ascending=False, inplace=True)
vars_keep = list(filter_score['variable'][1:num_filter+1])
print(filter_score.head(num_filter))
df_top = df_sampled.filter(vars_keep, axis=1)
del goods # delete these before starting the next file, if any
del bads
gc.collect()

filter_score.sort_values(by=['score'], ascending=False, inplace=True)

filter_score.head(30)

filter_score.tail(10)

filter_score.shape

filter_score.sort_values(by=['score'], ascending=False, inplace=True)
vars_keep = list(filter_score['variable'][num_files:num_filter+3])
print(1,' vars_keep:',vars_keep)

df_keep = df_top.filter(vars_keep, axis=1)
df_keep.head()

df_keep.shape

Y.head()

Y = Y.values
X = df_keep

print('time to here:', dt.datetime.now() - start_time)

print(X.shape,Y.shape)

def fdr(classifier, x, y, cutoff=detect_rate):
# Calculates FDR score for the given classifier on dataset x and y with cutoff value
# get the probability list from the given classifier
    return fdr_prob(y, classifier.predict_proba(x), cutoff)
def fdr_prob(y, y_prob, cutoff=detect_rate):
    if len(y_prob.shape) != 1:    # sometimes the proba list can contain many columns, one for each category
        y_prob = y_prob[:, -1:]   # only the last one (fraud_label==1) is used here.
    num_fraud = len(y[y == 1])    # count the total nunber of frauds   
# sort the proba list from high to low while retain the true (not predicted) fraud label
    sorted_prob = np.asarray(sorted(zip(y_prob, y), key=lambda x: x[0], reverse=True))
    cutoff_bin = sorted_prob[0:int(len(y) * cutoff), 1:]  # 3% cutoff
# return the FDR score (#fraud_in_cutoff / #total_fraud)
    return len(cutoff_bin[cutoff_bin == 1]) / num_fraud 

%%time
# If you're doing forward selection it's enough to stop at num_wrapper variables. 
# If you're doing backward selection you need to go through all the variables to get 
# a sorted list of num_wrapper variables.
nfeatures = len(X.columns)
clf = RandomForestClassifier(n_estimators=5, n_jobs=-1) # simple, fast nonlinear model for the wrapper
sfs = SFS(clf,k_features=num_wrapper,forward=True,verbose=2,scoring=fdr,cv=2,n_jobs=-1) # forward selection
# sfs = SFS(clf,k_features=1,forward=False,verbose=2,scoring=fdr,cv=2,n_jobs=-1) # backward selection
sfs.fit(X,Y)

from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
fig1 = plot_sfs(sfs.get_metric_dict(),kind='std_dev', figsize=(15, 6))
plt.xticks(np.arange(0, len(X.columns), step=5))
plt.ylim([0., .7])
plt.xlim(0,num_wrapper)
plt.title('Stepwise Selection')
plt.grid()
plt.savefig('FS_fdr.png')
plt.show()

vars_FS = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
vars_FS

vars_final = list(vars_FS.loc[num_wrapper, 'feature_names'])
vars_final

vars_FS.to_csv('VarsFS.csv', index=False)

sfs.subsets_

vars_final.append(index_name)
vars_fin mal.append(y_name)
vars_final

X_final = df_keep.filter(vars_final, axis=1)
X_final.shape

X_final.to_csv('vars_final.csv', index=False)

print('time to here:', dt.datetime.now() - start_time)

top30=[]
for i in vars_FS['feature_names']:
    for j in list(i):
        if j in top30:
            continue
        else:
            top30.append(j)
top30

import pandas as pd
file_name = "vars2.csv"
df = pd.read_csv(file_name)
df.shape

vars_final=['fulladdress_count_30',
 'ssn_lastname_count_30',
 'homephone_count_3',
 'ssn_count_14',
 'ssn_dob_count_0_by_30',
 'ssn_count_0_by_14',
 'fulladdress_homephone_count_7',
 'name_dob_count_14',
 'ssn_dob_count_7',
 'fulladdress_homephone_count_30',
 'ssn_count_0_by_30',
 'ssn_count_30',
 'ssn_count_7',
 'ssn_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_0_by_14',
 'ssn_name_dob_count_0_by_14',
 'ssn_name_dob_count_14',
 'name_dob_count_0_by_14',
 'ssn_dob_count_30',
 'name_dob_count_7',
 'ssn_name_dob_count_0_by_30',
 'name_dob_count_0_by_30',
 'ssn_dob_count_0_by_14',
 'ssn_firstname_count_7',
 'ssn_name_dob_count_30',
 'ssn_firstname_count_14',
 'ssn_lastname_count_0_by_14',
 'ssn_firstname_count_0_by_14',
 'name_dob_count_30',
 'record',
 'fraud_label']

X_final = df.filter(vars_final, axis=1)
X_final.shape

X_final.to_csv('vars_final.csv', index=False)